In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
from datetime import datetime

output_file = 'Softmax_results.txt'
dataset_path = './removed_bg_dataset'
image_size = (128, 128)

# Lists to store data
data = []
labels = []

# Load images from dataset
for breed in os.listdir(dataset_path):
    breed_path = os.path.join(dataset_path, breed)
    if os.path.isdir(breed_path):
        for img_name in os.listdir(breed_path):
            img_path = os.path.join(breed_path, img_name)
            try:
                # Ensure the file is a valid image
                with Image.open(img_path) as img:
                    img.verify()  # Check integrity
                
                # Read and preprocess the image
                image = cv2.imread(img_path)
                image = cv2.resize(image, image_size)  # Resize the image
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (reduces dimensions)
                data.append(image.flatten())  # Flatten the image into a vector
                labels.append(breed)
            except Exception as e:
                print(f"Skipping invalid image: {img_path}")

# Convert lists to NumPy arrays
data = np.array(data, dtype="float32") / 255.0  # Normalize pixel values
labels = np.array(labels)

# Encode class labels as integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)
X_train_final = X_train  # No transformation
X_test_final = X_test

# Train Softmax classifier
softmax_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
softmax_model.fit(X_train_final, y_train)

# Predictions
y_pred = softmax_model.predict(X_test_final)
accuracy = accuracy_score(y_test, y_pred)

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
details = f"Test Accuracy: {accuracy:.4f} image_size: {image_size} dataset: {dataset_path}"
output = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
results = f"{current_time} \n {details} \n {output}\n"
print(results)

with open(output_file, 'a') as f:
    f.write(f"Report generated at: {results}")
